# PortfolioLang - DSL para Carteiras de Investimentos
Trabalho de Linguagens Formais e Compiladores
Prof.: Ivan L. Süptitz

Grupo: João, Tiago e Victor Hugo


Este notebook implementa uma Domain Specific Language (DSL) para configuração
de carteiras de investimentos, incluindo analisador léxico, sintático e
gerador de código.


# PASSO 1: INSTALAÇÃO E IMPORTAÇÃO DAS BIBLIOTECAS

> Para executar este notebook, instale as dependências:

```
pip install pandas matplotlib datetime reportlab
```





In [16]:
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import reportlab
import os
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


# PASSO 2: CRIAÇÃO DA CLASSE TOKEN


In [17]:
@dataclass
class Token:
    """Representa um token da linguagem"""
    type: str
    value: Any
    line: int
    column: int

class TokenType:
    """Tipos de tokens da linguagem"""
    # Palavras-chave
    CARTEIRA = 'CARTEIRA'
    NOME = 'NOME'
    PERFIL = 'PERFIL'
    HORIZONTE_TEMPORAL = 'HORIZONTE_TEMPORAL'
    ALOCACAO = 'ALOCACAO'
    RESTRICOES = 'RESTRICOES'
    REBALANCEAMENTO = 'REBALANCEAMENTO'

    # Tipos de ativos
    ACOES_NACIONAIS = 'ACOES_NACIONAIS'
    ACOES_INTERNACIONAIS = 'ACOES_INTERNACIONAIS'
    FUNDOS_IMOBILIARIOS = 'FUNDOS_IMOBILIARIOS'
    FUNDOS_MULTIMERCADO = 'FUNDOS_MULTIMERCADO'
    RENDA_FIXA = 'RENDA_FIXA'

    # Parâmetros
    VOLATILIDADE_MAXIMA = 'VOLATILIDADE_MAXIMA'
    TAXA_ADMINISTRATIVA_MAXIMA = 'TAXA_ADMINISTRATIVA_MAXIMA'
    SETORIAL = 'SETORIAL'
    GEOGRAFICO = 'GEOGRAFICO'
    FREQUENCIA = 'FREQUENCIA'
    TOLERANCIA = 'TOLERANCIA'

    # Valores temporais
    ANOS = 'ANOS'
    MESES = 'MESES'
    TRIMESTRAL = 'TRIMESTRAL'
    SEMESTRAL = 'SEMESTRAL'
    ANUAL = 'ANUAL'
    MENSAL = 'MENSAL'

    # Símbolos
    IGUAL = 'IGUAL'
    CHAVE_ABRE = 'CHAVE_ABRE'
    CHAVE_FECHA = 'CHAVE_FECHA'
    PONTO_VIRGULA = 'PONTO_VIRGULA'
    PORCENTAGEM = 'PORCENTAGEM'

    # Literais
    STRING = 'STRING'
    NUMERO = 'NUMERO'
    IDENTIFICADOR = 'IDENTIFICADOR'

    # Especiais
    EOF = 'EOF'
    NEWLINE = 'NEWLINE'

# PASSO 3: CRIAÇÃO DO ANALISADOR LÉXICO MANUAL

In [18]:
class PortfolioLexer:
    """Analisador léxico manual para PortfolioLang"""

    def __init__(self):
        self.text = ""
        self.pos = 0
        self.line = 1
        self.column = 1
        self.tokens = []

        # Palavras reservadas
        self.keywords = {
            'carteira': TokenType.CARTEIRA,
            'nome': TokenType.NOME,
            'perfil': TokenType.PERFIL,
            'horizonte_temporal': TokenType.HORIZONTE_TEMPORAL,
            'alocação': TokenType.ALOCACAO,
            'restrições': TokenType.RESTRICOES,
            'rebalanceamento': TokenType.REBALANCEAMENTO,
            'ações_nacionais': TokenType.ACOES_NACIONAIS,
            'ações_internacionais': TokenType.ACOES_INTERNACIONAIS,
            'fundos_imobiliarios': TokenType.FUNDOS_IMOBILIARIOS,
            'fundos_multimercado': TokenType.FUNDOS_MULTIMERCADO,
            'renda_fixa': TokenType.RENDA_FIXA,
            'volatilidade_maxima': TokenType.VOLATILIDADE_MAXIMA,
            'taxa_administrativa_maxima': TokenType.TAXA_ADMINISTRATIVA_MAXIMA,
            'setorial': TokenType.SETORIAL,
            'geografico': TokenType.GEOGRAFICO,
            'frequencia': TokenType.FREQUENCIA,
            'tolerancia': TokenType.TOLERANCIA,
            'anos': TokenType.ANOS,
            'meses': TokenType.MESES,
            'trimestral': TokenType.TRIMESTRAL,
            'semestral': TokenType.SEMESTRAL,
            'anual': TokenType.ANUAL,
            'mensal': TokenType.MENSAL
        }

    def current_char(self):
        """Retorna o caractere atual"""
        if self.pos >= len(self.text):
            return None
        return self.text[self.pos]

    def peek_char(self, offset=1):
        """Olha para frente sem consumir"""
        peek_pos = self.pos + offset
        if peek_pos >= len(self.text):
            return None
        return self.text[peek_pos]

    def advance(self):
        """Avança uma posição"""
        if self.pos < len(self.text) and self.text[self.pos] == '\n':
            self.line += 1
            self.column = 1
        else:
            self.column += 1
        self.pos += 1

    def skip_whitespace(self):
        """Pula espaços em branco"""
        while self.current_char() and self.current_char() in ' \t':
            self.advance()

    def skip_newlines(self):
        """Pula quebras de linha"""
        while self.current_char() and self.current_char() in '\n\r':
            self.advance()

    def read_string(self):
        """Lê uma string entre aspas"""
        value = ""
        self.advance()  # Pula a aspa inicial

        while self.current_char() and self.current_char() != '"':
            value += self.current_char()
            self.advance()

        if self.current_char() == '"':
            self.advance()  # Pula a aspa final
        else:
            raise Exception(f"String não fechada na linha {self.line}")

        return value

    def read_number(self):
        """Lê um número (inteiro ou decimal)"""
        value = ""

        while self.current_char() and (self.current_char().isdigit() or self.current_char() == '.'):
            value += self.current_char()
            self.advance()

        # Converter para int ou float
        if '.' in value:
            return float(value)
        else:
            return int(value)

    def read_identifier(self):
        """Lê um identificador ou palavra-chave"""
        value = ""

        while self.current_char() and (self.current_char().isalnum() or self.current_char() in '_ã'):
            value += self.current_char()
            self.advance()

        return value

    def tokenize(self, text):
        """Tokeniza o texto de entrada"""
        self.text = text
        self.pos = 0
        self.line = 1
        self.column = 1
        self.tokens = []

        while self.current_char():
            # Pular espaços
            if self.current_char() in ' \t':
                self.skip_whitespace()
                continue

            # Quebras de linha
            if self.current_char() in '\n\r':
                self.skip_newlines()
                continue

            # Símbolos simples
            if self.current_char() == '=':
                self.tokens.append(Token(TokenType.IGUAL, '=', self.line, self.column))
                self.advance()
            elif self.current_char() == '{':
                self.tokens.append(Token(TokenType.CHAVE_ABRE, '{', self.line, self.column))
                self.advance()
            elif self.current_char() == '}':
                self.tokens.append(Token(TokenType.CHAVE_FECHA, '}', self.line, self.column))
                self.advance()
            elif self.current_char() == ';':
                self.tokens.append(Token(TokenType.PONTO_VIRGULA, ';', self.line, self.column))
                self.advance()
            elif self.current_char() == '%':
                self.tokens.append(Token(TokenType.PORCENTAGEM, '%', self.line, self.column))
                self.advance()

            # Strings
            elif self.current_char() == '"':
                string_value = self.read_string()
                self.tokens.append(Token(TokenType.STRING, string_value, self.line, self.column))

            # Números
            elif self.current_char().isdigit():
                number_value = self.read_number()
                self.tokens.append(Token(TokenType.NUMERO, number_value, self.line, self.column))

            # Identificadores e palavras-chave
            elif self.current_char().isalpha() or self.current_char() == '_':
                identifier = self.read_identifier()
                token_type = self.keywords.get(identifier, TokenType.IDENTIFICADOR)
                self.tokens.append(Token(token_type, identifier, self.line, self.column))

            else:
                raise Exception(f"Caractere inesperado '{self.current_char()}' na linha {self.line}, coluna {self.column}")

        # Adicionar EOF
        self.tokens.append(Token(TokenType.EOF, None, self.line, self.column))
        return self.tokens


# PASSO 4: CRIAÇÃO DO ANALISADOR SINTÁTICO MANUAL

In [19]:
class PortfolioParser:
    """Analisador sintático manual usando recursive descent parsing"""

    def __init__(self):
        self.tokens = []
        self.pos = 0
        self.current_token = None
        self.errors = []

    def peek_token(self, offset=0):
        """Olha o token atual ou com offset"""
        peek_pos = self.pos + offset
        if peek_pos >= len(self.tokens):
            return self.tokens[-1]  # EOF
        return self.tokens[peek_pos]

    def advance_token(self):
        """Avança para o próximo token"""
        if self.pos < len(self.tokens) - 1:
            self.pos += 1
        self.current_token = self.tokens[self.pos]

    def expect_token(self, expected_type):
        """Espera um token específico"""
        if self.current_token.type != expected_type:
            raise Exception(f"Esperado {expected_type}, encontrado {self.current_token.type} na linha {self.current_token.line}")
        value = self.current_token.value
        self.advance_token()
        return value

    def parse(self, tokens):
        """Parse principal"""
        self.tokens = tokens
        self.pos = 0
        self.current_token = tokens[0]
        self.errors = []

        try:
            result = self.parse_programa()
            return result
        except Exception as e:
            self.errors.append(str(e))
            print(f"❌ Erro de parsing: {e}")
            return None

    def parse_programa(self):
        """programa : carteira"""
        return self.parse_carteira()

    def parse_carteira(self):
        """carteira : CARTEIRA '{' configuracoes alocacao restricoes? rebalanceamento? '}'"""
        self.expect_token(TokenType.CARTEIRA)
        self.expect_token(TokenType.CHAVE_ABRE)

        configuracoes = self.parse_configuracoes()
        alocacao = self.parse_alocacao()

        # Seções opcionais
        restricoes = {}
        rebalanceamento = {}

        if self.current_token.type == TokenType.RESTRICOES:
            restricoes = self.parse_restricoes()

        if self.current_token.type == TokenType.REBALANCEAMENTO:
            rebalanceamento = self.parse_rebalanceamento()

        self.expect_token(TokenType.CHAVE_FECHA)

        return {
            'configuracoes': configuracoes,
            'alocacao': alocacao,
            'restricoes': restricoes,
            'rebalanceamento': rebalanceamento
        }

    def parse_configuracoes(self):
        """Parse das configurações da carteira"""
        configuracoes = {}

        # Processar configurações até encontrar alocação
        while self.current_token.type in [TokenType.NOME, TokenType.PERFIL, TokenType.HORIZONTE_TEMPORAL]:
            if self.current_token.type == TokenType.NOME:
                self.advance_token()
                self.expect_token(TokenType.IGUAL)
                nome = self.expect_token(TokenType.STRING)
                self.expect_token(TokenType.PONTO_VIRGULA)
                configuracoes['nome'] = nome

            elif self.current_token.type == TokenType.PERFIL:
                self.advance_token()
                self.expect_token(TokenType.IGUAL)
                perfil = self.expect_token(TokenType.STRING)
                self.expect_token(TokenType.PONTO_VIRGULA)
                configuracoes['perfil'] = perfil

            elif self.current_token.type == TokenType.HORIZONTE_TEMPORAL:
                self.advance_token()
                self.expect_token(TokenType.IGUAL)
                numero = self.expect_token(TokenType.NUMERO)
                unidade = self.current_token.value
                if self.current_token.type in [TokenType.ANOS, TokenType.MESES]:
                    self.advance_token()
                else:
                    raise Exception(f"Esperado 'anos' ou 'meses', encontrado {self.current_token.type}")
                self.expect_token(TokenType.PONTO_VIRGULA)
                configuracoes['horizonte_temporal'] = f"{numero} {unidade}"

        return configuracoes

    def parse_alocacao(self):
        """Parse da seção de alocação"""
        self.expect_token(TokenType.ALOCACAO)
        self.expect_token(TokenType.CHAVE_ABRE)

        alocacao = {}

        # Processar alocações de ativos
        asset_types = [
            TokenType.ACOES_NACIONAIS, TokenType.ACOES_INTERNACIONAIS,
            TokenType.FUNDOS_IMOBILIARIOS, TokenType.FUNDOS_MULTIMERCADO,
            TokenType.RENDA_FIXA
        ]

        while self.current_token.type in asset_types:
            asset_type = self.current_token.value
            self.advance_token()
            self.expect_token(TokenType.IGUAL)
            percentage = self.expect_token(TokenType.NUMERO)
            self.expect_token(TokenType.PORCENTAGEM)
            self.expect_token(TokenType.PONTO_VIRGULA)

            alocacao[asset_type] = percentage
            print(f"  📈 {asset_type}: {percentage}%")

        self.expect_token(TokenType.CHAVE_FECHA)
        print(f"💰 Alocação processada: {len(alocacao)} ativos")

        return alocacao

    def parse_restricoes(self):
        """Parse da seção de restrições"""
        self.expect_token(TokenType.RESTRICOES)
        self.expect_token(TokenType.CHAVE_ABRE)

        restricoes = {}

        # Processar restrições
        while self.current_token.type in [TokenType.VOLATILIDADE_MAXIMA, TokenType.TAXA_ADMINISTRATIVA_MAXIMA,
                                        TokenType.SETORIAL, TokenType.GEOGRAFICO]:

            if self.current_token.type == TokenType.VOLATILIDADE_MAXIMA:
                self.advance_token()
                self.expect_token(TokenType.IGUAL)
                valor = self.expect_token(TokenType.NUMERO)
                self.expect_token(TokenType.PORCENTAGEM)
                self.expect_token(TokenType.PONTO_VIRGULA)
                restricoes['volatilidade_maxima'] = valor

            elif self.current_token.type == TokenType.TAXA_ADMINISTRATIVA_MAXIMA:
                self.advance_token()
                self.expect_token(TokenType.IGUAL)
                valor = self.expect_token(TokenType.NUMERO)
                self.expect_token(TokenType.PORCENTAGEM)
                self.expect_token(TokenType.PONTO_VIRGULA)
                restricoes['taxa_administrativa_maxima'] = valor

            elif self.current_token.type == TokenType.SETORIAL:
                restricoes['setorial'] = self.parse_restricao_complexa(TokenType.SETORIAL)

            elif self.current_token.type == TokenType.GEOGRAFICO:
                restricoes['geografico'] = self.parse_restricao_complexa(TokenType.GEOGRAFICO)

        self.expect_token(TokenType.CHAVE_FECHA)
        print("🔒 Restrições processadas")

        return restricoes

    def parse_restricao_complexa(self, tipo):
        """Parse de restrições setoriais ou geográficas"""
        self.advance_token()  # Consome SETORIAL ou GEOGRAFICO
        self.expect_token(TokenType.CHAVE_ABRE)

        restricao = {}

        while self.current_token.type == TokenType.IDENTIFICADOR:
            nome = self.expect_token(TokenType.IDENTIFICADOR)
            self.expect_token(TokenType.IGUAL)
            valor = self.expect_token(TokenType.NUMERO)
            self.expect_token(TokenType.PORCENTAGEM)
            self.expect_token(TokenType.PONTO_VIRGULA)
            restricao[nome] = valor

        self.expect_token(TokenType.CHAVE_FECHA)
        return restricao

    def parse_rebalanceamento(self):
        """Parse da seção de rebalanceamento"""
        self.expect_token(TokenType.REBALANCEAMENTO)
        self.expect_token(TokenType.CHAVE_ABRE)

        rebalanceamento = {}

        # Frequência
        if self.current_token.type == TokenType.FREQUENCIA:
            self.advance_token()
            self.expect_token(TokenType.IGUAL)
            frequencia = self.current_token.value
            if self.current_token.type in [TokenType.TRIMESTRAL, TokenType.SEMESTRAL, TokenType.ANUAL, TokenType.MENSAL]:
                self.advance_token()
            else:
                raise Exception("Frequência inválida")
            self.expect_token(TokenType.PONTO_VIRGULA)
            rebalanceamento['frequencia'] = frequencia

        # Tolerância
        if self.current_token.type == TokenType.TOLERANCIA:
            self.advance_token()
            self.expect_token(TokenType.IGUAL)
            tolerancia = self.expect_token(TokenType.NUMERO)
            self.expect_token(TokenType.PORCENTAGEM)
            self.expect_token(TokenType.PONTO_VIRGULA)
            rebalanceamento['tolerancia'] = tolerancia

        self.expect_token(TokenType.CHAVE_FECHA)
        print("⚖️ Rebalanceamento processado")

        return rebalanceamento

# PASSO 5: CRIAÇÃO DO VALIDADOR SEMÂNTICO

In [20]:
class PortfolioValidator:
    """Validador semântico para verificar consistência da carteira"""

    def __init__(self):
        self.warnings = []
        self.errors = []

    def validate(self, portfolio_data):
        """Executa todas as validações semânticas"""
        self.warnings = []
        self.errors = []

        print("\n🔍 Executando validações semânticas...")

        # Validação 1: Soma das alocações
        self._validate_allocation_sum(portfolio_data)

        # Validação 2: Percentuais válidos
        self._validate_percentage_ranges(portfolio_data)

        # Validação 3: Consistência de perfil
        self._validate_risk_profile_consistency(portfolio_data)

        return len(self.errors) == 0

    def _validate_allocation_sum(self, data):
        """Verifica se a soma das alocações é 100%"""
        if 'alocacao' in data:
            total = sum(data['alocacao'].values())
            if abs(total - 100) > 0.01:
                self.errors.append(f"Soma das alocações é {total}%, deve ser 100%")
            else:
                print("✅ Soma das alocações: 100%")

    def _validate_percentage_ranges(self, data):
        """Verifica se todos os percentuais estão no intervalo válido"""
        def check_range(value, name):
            if not (0 <= value <= 100):
                self.errors.append(f"{name}: {value}% fora do intervalo [0, 100]")

        if 'alocacao' in data:
            for asset, percentage in data['alocacao'].items():
                check_range(percentage, f"Alocação {asset}")

        print("✅ Intervalos percentuais validados")

    def _validate_risk_profile_consistency(self, data):
        """Verifica consistência entre perfil de risco e alocação"""
        if 'configuracoes' not in data or 'perfil' not in data['configuracoes']:
            return

        profile = data['configuracoes']['perfil'].lower()
        alocacao = data.get('alocacao', {})

        high_risk_assets = ['ações_nacionais', 'ações_internacionais', 'fundos_multimercado']
        risk_exposure = sum(alocacao.get(asset, 0) for asset in high_risk_assets)

        if profile == 'conservador' and risk_exposure > 30:
            self.warnings.append(f"Perfil conservador com {risk_exposure}% em ativos de risco")
        elif profile == 'moderado' and (risk_exposure < 20 or risk_exposure > 70):
            self.warnings.append(f"Perfil moderado com {risk_exposure}% em ativos de risco")
        elif profile == 'arrojado' and risk_exposure < 50:
            self.warnings.append(f"Perfil arrojado com apenas {risk_exposure}% em ativos de risco")

        print(f"✅ Perfil de risco '{profile}' com {risk_exposure}% em ativos de risco")

    def print_results(self):
        """Imprime resultados das validações"""
        if self.errors:
            print("\n❌ ERROS ENCONTRADOS:")
            for error in self.errors:
                print(f"  • {error}")

        if self.warnings:
            print("\n⚠️ AVISOS:")
            for warning in self.warnings:
                print(f"  • {warning}")

        if not self.errors and not self.warnings:
            print("\n✅ Todas as validações passaram!")


# PASSO 6: GERADOR DE PDF

In [21]:
try:
    from reportlab.lib.pagesizes import letter, A4
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.lib import colors
    from reportlab.graphics.shapes import Drawing
    from reportlab.graphics.charts.piecharts import Pie
    from reportlab.lib.colors import HexColor
    PDF_AVAILABLE = True
    print("✅ ReportLab disponível - PDFs podem ser gerados")
except ImportError:
    PDF_AVAILABLE = False
    print("⚠️ ReportLab não disponível - execute: pip install reportlab")

class PortfolioCodeGenerator:
    """Gerador de código, relatórios e PDFs"""

    def __init__(self, portfolio_data):
        self.data = portfolio_data
        self.timestamp = datetime.now()

    def generate_python_code(self):
        """Gera código Python para análise da carteira"""
        config = self.data.get('configuracoes', {})
        alocacao = self.data.get('alocacao', {})

        code = f'''# Código gerado automaticamente - PortfolioLang
# Data: {self.timestamp.strftime("%Y-%m-%d %H:%M:%S")}

import pandas as pd
import matplotlib.pyplot as plt

class Portfolio:
    """Classe gerada para análise da carteira: {config.get('nome', 'Sem nome')}"""

    def __init__(self):
        # Configurações da carteira
        self.config = {json.dumps(config, indent=12, ensure_ascii=False)}

        # Alocação de ativos (percentual)
        self.allocation = {json.dumps(alocacao, indent=12, ensure_ascii=False)}

        # Restrições
        self.restrictions = {json.dumps(self.data.get('restricoes', {}), indent=12, ensure_ascii=False)}

        # Configurações de rebalanceamento
        self.rebalancing = {json.dumps(self.data.get('rebalanceamento', {}), indent=12, ensure_ascii=False)}

    def get_allocation_dataframe(self):
        """Retorna DataFrame com a alocação"""
        return pd.DataFrame(list(self.allocation.items()),
                          columns=['Ativo', 'Percentual'])

    def plot_allocation(self):
        """Gera gráfico de pizza da alocação"""
        df = self.get_allocation_dataframe()
        plt.figure(figsize=(10, 8))
        plt.pie(df['Percentual'], labels=df['Ativo'], autopct='%1.1f%%')
        plt.title(f"Alocação da Carteira: {{self.config.get('nome', 'Sem nome')}}")
        plt.show()

    def calculate_risk_exposure(self):
        """Calcula exposição ao risco"""
        high_risk_assets = ['ações_nacionais', 'ações_internacionais', 'fundos_multimercado']
        risk_exposure = sum(self.allocation.get(asset, 0) for asset in high_risk_assets)
        return risk_exposure

    def get_summary(self):
        """Retorna resumo da carteira"""
        return {{
            'nome': self.config.get('nome'),
            'perfil': self.config.get('perfil'),
            'horizonte': self.config.get('horizonte_temporal'),
            'total_ativos': len(self.allocation),
            'exposicao_risco': self.calculate_risk_exposure()
        }}

# Instancia da carteira
portfolio = Portfolio()

# Exemplo de uso
if __name__ == "__main__":
    print("📊 Resumo da Carteira:")
    summary = portfolio.get_summary()
    for key, value in summary.items():
        print(f"  {{key.replace('_', ' ').title()}}: {{value}}")

    print("\\n📈 Alocação de Ativos:")
    df = portfolio.get_allocation_dataframe()
    print(df.to_string(index=False))

    print(f"\\n🎯 Exposição ao Risco: {{portfolio.calculate_risk_exposure()}}%")
'''
        return code

    def generate_pdf_report(self, filename=None):
        """Gera um relatório PDF bonito da carteira"""

        if not PDF_AVAILABLE:
            print("❌ ReportLab não disponível. Execute: pip install reportlab")
            return None

        # Nome do arquivo
        if not filename:
            config = self.data.get('configuracoes', {})
            nome_carteira = config.get('nome', 'carteira')
            safe_name = "".join(c for c in nome_carteira if c.isalnum() or c in (' ', '-', '_')).strip()
            safe_name = safe_name.replace(' ', '_').lower()
            filename = f"{safe_name}_relatorio.pdf"

        try:
            # Criar documento
            doc = SimpleDocTemplate(filename, pagesize=A4,
                                  rightMargin=72, leftMargin=72,
                                  topMargin=72, bottomMargin=18)

            # Elementos do relatório
            story = []
            styles = getSampleStyleSheet()

            # Estilos customizados
            title_style = ParagraphStyle(
                'CustomTitle',
                parent=styles['Heading1'],
                fontSize=24,
                spaceAfter=30,
                textColor=colors.darkblue,
                alignment=1  # Centralizado
            )

            heading_style = ParagraphStyle(
                'CustomHeading',
                parent=styles['Heading2'],
                fontSize=16,
                spaceAfter=12,
                textColor=colors.darkblue,
                borderWidth=1,
                borderColor=colors.darkblue,
                borderPadding=5
            )

            normal_style = ParagraphStyle(
                'CustomNormal',
                parent=styles['Normal'],
                fontSize=12,
                spaceAfter=6
            )

            # ===== CABEÇALHO =====
            story.append(Paragraph("📊 RELATÓRIO DE CARTEIRA DE INVESTIMENTOS", title_style))
            story.append(Spacer(1, 20))

            # Data de geração
            data_geracao = self.timestamp.strftime("%d/%m/%Y às %H:%M:%S")
            story.append(Paragraph(f"<i>Gerado em: {data_geracao}</i>", styles['Normal']))
            story.append(Paragraph("<i>Sistema: PortfolioLang DSL</i>", styles['Normal']))
            story.append(Spacer(1, 30))

            # ===== INFORMAÇÕES GERAIS =====
            story.append(Paragraph("ℹ️ INFORMAÇÕES GERAIS", heading_style))
            story.append(Spacer(1, 10))

            config = self.data.get('configuracoes', {})
            info_data = [
                ['Nome da Carteira:', config.get('nome', 'Não informado')],
                ['Perfil de Risco:', config.get('perfil', 'Não informado').title()],
                ['Horizonte Temporal:', config.get('horizonte_temporal', 'Não informado')],
            ]

            info_table = Table(info_data, colWidths=[2*inch, 3*inch])
            info_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (0, -1), colors.lightblue),
                ('TEXTCOLOR', (0, 0), (0, -1), colors.darkblue),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
                ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 12),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ]))

            story.append(info_table)
            story.append(Spacer(1, 30))

            # ===== ALOCAÇÃO DE ATIVOS =====
            story.append(Paragraph("💰 ALOCAÇÃO DE ATIVOS", heading_style))
            story.append(Spacer(1, 10))

            alocacao = self.data.get('alocacao', {})

            # Tabela de alocação
            alocacao_data = [['Classe de Ativo', 'Percentual (%)', 'Classificação de Risco']]

            high_risk_assets = ['ações_nacionais', 'ações_internacionais', 'fundos_multimercado']

            for asset, percentage in alocacao.items():
                asset_display = asset.replace('_', ' ').title()
                risk_class = "Alto Risco" if asset in high_risk_assets else "Baixo Risco"
                alocacao_data.append([asset_display, f"{percentage}%", risk_class])

            # Adicionar total
            total_allocation = sum(alocacao.values())
            alocacao_data.append(['TOTAL', f"{total_allocation}%", ""])

            alocacao_table = Table(alocacao_data, colWidths=[2.5*inch, 1*inch, 1.5*inch])
            alocacao_table.setStyle(TableStyle([
                # Cabeçalho
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 12),

                # Dados
                ('FONTNAME', (0, 1), (-1, -2), 'Helvetica'),
                ('FONTSIZE', (0, 1), (-1, -2), 11),
                ('ALIGN', (1, 0), (1, -1), 'CENTER'),
                ('ALIGN', (2, 0), (2, -1), 'CENTER'),

                # Total
                ('BACKGROUND', (0, -1), (-1, -1), colors.lightgrey),
                ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold'),

                # Bordas
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),

                # Cores alternadas para legibilidade
                ('BACKGROUND', (0, 1), (-1, -2), colors.beige),
            ]))

            # Alternar cores das linhas
            for i in range(1, len(alocacao_data) - 1, 2):
                alocacao_table.setStyle(TableStyle([
                    ('BACKGROUND', (0, i), (-1, i), colors.white),
                ]))

            story.append(alocacao_table)
            story.append(Spacer(1, 30))

            # ===== MÉTRICAS DA CARTEIRA =====
            story.append(Paragraph("📈 MÉTRICAS DA CARTEIRA", heading_style))
            story.append(Spacer(1, 10))

            # Calcular métricas
            total_ativos = len(alocacao)
            risk_exposure = sum(alocacao.get(asset, 0) for asset in high_risk_assets)
            conservative_exposure = 100 - risk_exposure

            metricas_data = [
                ['Total de Classes de Ativos:', str(total_ativos)],
                ['Exposição a Alto Risco:', f"{risk_exposure}%"],
                ['Exposição Conservadora:', f"{conservative_exposure}%"],
                ['Diversificação:', 'Alta' if total_ativos >= 4 else 'Média' if total_ativos >= 2 else 'Baixa'],
            ]

            # Análise do perfil
            profile = config.get('perfil', '').lower()
            if profile == 'conservador' and risk_exposure > 30:
                profile_analysis = "⚠️ Alto risco para perfil conservador"
            elif profile == 'arrojado' and risk_exposure < 50:
                profile_analysis = "⚠️ Baixo risco para perfil arrojado"
            else:
                profile_analysis = "✅ Alocação adequada ao perfil"

            metricas_data.append(['Análise do Perfil:', profile_analysis])

            metricas_table = Table(metricas_data, colWidths=[2.5*inch, 2.5*inch])
            metricas_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (0, -1), colors.lightgreen),
                ('TEXTCOLOR', (0, 0), (0, -1), colors.darkgreen),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
                ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 11),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ]))

            story.append(metricas_table)
            story.append(Spacer(1, 30))

            # ===== RESTRIÇÕES =====
            restricoes = self.data.get('restricoes', {})
            if restricoes:
                story.append(Paragraph("🔒 RESTRIÇÕES E LIMITES", heading_style))
                story.append(Spacer(1, 10))

                restricoes_data = []

                # Restrições simples
                if 'volatilidade_maxima' in restricoes:
                    restricoes_data.append(['Volatilidade Máxima:', f"{restricoes['volatilidade_maxima']}%"])

                if 'taxa_administrativa_maxima' in restricoes:
                    restricoes_data.append(['Taxa Administrativa Máxima:', f"{restricoes['taxa_administrativa_maxima']}%"])

                # Restrições setoriais
                if 'setorial' in restricoes:
                    story.append(Paragraph("<b>Limites Setoriais:</b>", normal_style))
                    for setor, limite in restricoes['setorial'].items():
                        restricoes_data.append([f"  {setor.title()}:", f"{limite}%"])

                # Restrições geográficas
                if 'geografico' in restricoes:
                    story.append(Paragraph("<b>Limites Geográficos:</b>", normal_style))
                    for regiao, limite in restricoes['geografico'].items():
                        restricoes_data.append([f"  {regiao.title()}:", f"{limite}%"])

                if restricoes_data:
                    restricoes_table = Table(restricoes_data, colWidths=[2.5*inch, 2*inch])
                    restricoes_table.setStyle(TableStyle([
                        ('BACKGROUND', (0, 0), (0, -1), colors.lightyellow),
                        ('TEXTCOLOR', (0, 0), (0, -1), colors.darkorange),
                        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
                        ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
                        ('FONTSIZE', (0, 0), (-1, -1), 11),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black),
                        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                    ]))

                    story.append(restricoes_table)
                    story.append(Spacer(1, 30))

            # ===== REBALANCEAMENTO =====
            rebalanceamento = self.data.get('rebalanceamento', {})
            if rebalanceamento:
                story.append(Paragraph("⚖️ CONFIGURAÇÕES DE REBALANCEAMENTO", heading_style))
                story.append(Spacer(1, 10))

                rebal_data = []
                if 'frequencia' in rebalanceamento:
                    rebal_data.append(['Frequência:', rebalanceamento['frequencia'].title()])

                if 'tolerancia' in rebalanceamento:
                    rebal_data.append(['Tolerância:', f"{rebalanceamento['tolerancia']}%"])

                if rebal_data:
                    rebal_table = Table(rebal_data, colWidths=[2*inch, 2*inch])
                    rebal_table.setStyle(TableStyle([
                        ('BACKGROUND', (0, 0), (0, -1), colors.lightcyan),
                        ('TEXTCOLOR', (0, 0), (0, -1), colors.darkblue),
                        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
                        ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
                        ('FONTSIZE', (0, 0), (-1, -1), 11),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black),
                        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                    ]))

                    story.append(rebal_table)
                    story.append(Spacer(1, 30))

            # ===== GRÁFICO DE ALOCAÇÃO (se matplotlib disponível) =====
            try:
                # Criar gráfico de pizza simples em texto
                story.append(Paragraph("📊 DISTRIBUIÇÃO DA ALOCAÇÃO", heading_style))
                story.append(Spacer(1, 10))

                # Representação visual em texto
                vis_data = [['Ativo', 'Percentual', 'Representação Visual']]

                for asset, percentage in sorted(alocacao.items(), key=lambda x: x[1], reverse=True):
                    asset_display = asset.replace('_', ' ').title()
                    # Criar barra visual simples
                    bar_length = int(percentage / 5)  # 1 caractere para cada 5%
                    visual_bar = "█" * bar_length + "░" * (20 - bar_length)
                    vis_data.append([asset_display, f"{percentage}%", visual_bar])

                vis_table = Table(vis_data, colWidths=[2*inch, 1*inch, 2.5*inch])
                vis_table.setStyle(TableStyle([
                    # Cabeçalho
                    ('BACKGROUND', (0, 0), (-1, 0), colors.purple),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 11),

                    # Dados
                    ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
                    ('FONTSIZE', (0, 1), (-1, -1), 10),
                    ('FONTNAME', (2, 1), (2, -1), 'Courier'),  # Fonte monospace para barras
                    ('ALIGN', (1, 0), (1, -1), 'CENTER'),
                    ('ALIGN', (2, 0), (2, -1), 'LEFT'),

                    # Bordas
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ]))

                story.append(vis_table)
                story.append(Spacer(1, 30))

            except Exception as e:
                print(f"⚠️ Não foi possível gerar gráfico: {e}")

            # ===== RODAPÉ =====
            story.append(Spacer(1, 50))

            footer_style = ParagraphStyle(
                'Footer',
                parent=styles['Normal'],
                fontSize=9,
                textColor=colors.grey,
                alignment=1  # Centralizado
            )

            story.append(Paragraph("_" * 80, footer_style))
            story.append(Spacer(1, 10))
            story.append(Paragraph("Relatório gerado automaticamente pelo PortfolioLang DSL", footer_style))
            story.append(Paragraph("Prof. Ivan L. Süptitz - Linguagens Formais e Compiladores", footer_style))
            story.append(Paragraph("Universidade de Santa Cruz do Sul - UNISC", footer_style))

            # ===== GERAR PDF =====
            doc.build(story)

            print(f"✅ PDF gerado com sucesso: {filename}")
            print(f"📄 Tamanho: {os.path.getsize(filename) / 1024:.1f} KB")

            return filename

        except Exception as e:
            print(f"❌ Erro ao gerar PDF: {e}")
            return None

✅ ReportLab disponível - PDFs podem ser gerados


# PASSO 7 (FINAL) - EXECUÇÃO DO CÓDIGO

In [22]:
def test_portfolio_dsl():
    """Teste principal da DSL manual"""

    print("="*80)
    print("🚀 TESTANDO PORTFOLIOLANG DSL - VERSÃO MANUAL")
    print("="*80)

    # Código de teste
    portfolio_code = '''
    carteira {
        nome = "Carteira Diversificada";
        perfil = "moderado";
        horizonte_temporal = 5 anos;

        alocação {
            ações_nacionais = 30%;
            ações_internacionais = 20%;
            fundos_imobiliarios = 15%;
            fundos_multimercado = 10%;
            renda_fixa = 25%;
        }

        restrições {
            volatilidade_maxima = 10%;
            taxa_administrativa_maxima = 2%;
        }

        rebalanceamento {
            frequencia = trimestral;
            tolerancia = 2%;
        }
    }
    '''

    print("📝 Código de entrada:")
    print(portfolio_code)

    # Passo 1: Análise Léxica
    print("\n" + "="*60)
    print("🔍 PASSO 1: ANÁLISE LÉXICA")
    print("="*60)

    lexer = PortfolioLexer()
    try:
        tokens = lexer.tokenize(portfolio_code)
        print(f"✅ Análise léxica concluída! {len(tokens)} tokens encontrados")

        print("\nPrimeiros 15 tokens:")
        for i, token in enumerate(tokens[:15]):
            print(f"  {i+1:2d}. {token.type:20} -> {token.value}")

        if len(tokens) > 15:
            print(f"  ... e mais {len(tokens) - 15} tokens")

    except Exception as e:
        print(f"❌ Erro na análise léxica: {e}")
        return False

    # Passo 2: Análise Sintática
    print("\n" + "="*60)
    print("🔧 PASSO 2: ANÁLISE SINTÁTICA")
    print("="*60)

    parser = PortfolioParser()
    try:
        result = parser.parse(tokens)

        if result:
            print("✅ Análise sintática concluída!")
            print("\n📊 Estrutura gerada:")
            print(json.dumps(result, indent=2, ensure_ascii=False))
        else:
            print("❌ Falha na análise sintática!")
            return False

    except Exception as e:
        print(f"❌ Erro na análise sintática: {e}")
        return False

    # Passo 3: Validação Semântica
    print("\n" + "="*60)
    print("🔍 PASSO 3: VALIDAÇÃO SEMÂNTICA")
    print("="*60)

    validator = PortfolioValidator()
    is_valid = validator.validate(result)
    validator.print_results()

    # Passo 4: Geração de Código
    print("\n" + "="*60)
    print("⚙️ PASSO 4: GERAÇÃO DE CÓDIGO")
    print("="*60)

    generator = PortfolioCodeGenerator(result)

    # Gerar código Python
    python_code = generator.generate_python_code()
    print("📋 Código Python gerado:")
    print(python_code[:500] + "..." if len(python_code) > 500 else python_code)

    # Gerar PDF
    print("\n📄 Gerando relatório PDF...")
    pdf_filename = generator.generate_pdf_report()

    if pdf_filename:
        print(f"✅ PDF salvo como: {pdf_filename}")
        print("📖 Abra o arquivo para ver o relatório completo!")
    else:
        print("⚠️ PDF não pôde ser gerado. Gerando relatório em texto...")
        # Fallback para relatório em texto
        report = generator.generate_report()
        print("\n📄 Relatório em texto:")
        print(report)

    print("\n" + "="*60)
    print("✅ TESTE COMPLETO FINALIZADO COM SUCESSO!")
    print("="*60)

    return True, result, generator

# ===============================================================================
# EXECUÇÃO AUTOMÁTICA
# ===============================================================================

print("\n🎯 PortfolioLang DSL ")
print("="*50)

# Executar teste automaticamente
success, result, generator = test_portfolio_dsl()

if success:
    print("\n🎉 DSL funcionando perfeitamente!")
    print("📚 Agora você pode:")
    print("  - Modificar o código da carteira")
    print("  - Testar com seus próprios exemplos")
    print("  - Usar as funções individuais")
    print("  - Gerar relatórios customizados")
else:
    print("\n❌ Houve problemas na execução")


🎯 PortfolioLang DSL 
🚀 TESTANDO PORTFOLIOLANG DSL - VERSÃO MANUAL
📝 Código de entrada:

    carteira {
        nome = "Carteira Diversificada";
        perfil = "moderado";
        horizonte_temporal = 5 anos;

        alocação {
            ações_nacionais = 30%;
            ações_internacionais = 20%;
            fundos_imobiliarios = 15%;
            fundos_multimercado = 10%;
            renda_fixa = 25%;
        }

        restrições {
            volatilidade_maxima = 10%;
            taxa_administrativa_maxima = 2%;
        }

        rebalanceamento {
            frequencia = trimestral;
            tolerancia = 2%;
        }
    }
    

🔍 PASSO 1: ANÁLISE LÉXICA
✅ Análise léxica concluída! 70 tokens encontrados

Primeiros 15 tokens:
   1. CARTEIRA             -> carteira
   2. CHAVE_ABRE           -> {
   3. NOME                 -> nome
   4. IGUAL                -> =
   5. STRING               -> Carteira Diversificada
   6. PONTO_VIRGULA        -> ;
   7. PERFIL            